Pollenate TO Dataset

In [1]:
import geopandas as gpd

# Path to your GeoJSON file
geojson_file_path = 'PollinateTO Primary Project Garden Locations - 4326.geojson'

# Read the GeoJSON file
data = gpd.read_file(geojson_file_path)



In [ ]:
# Display the first few rows
data.head()

,_id,ID,YEAR_FUNDED,GROUP_NAME,PROJECT_NAME,GARDEN_TYPE,NUMBER_OF_GARDENS,ESTIMATED_GARDEN_SIZE,NEIGHBOURHOOD_NUMBER,NIA_OR_EN,PRIMARY_LOCATION_NAME,PRIMARY_GARDEN_ADDRESS,POSTAL_CODE,IS_INDIGENOUS_GARDEN,WARD_NAME,WARD_NUMBER,DESCRIPTION,IMAGE_NAME,IMAGE_ALT_TEXT,geometry
0,1,1,2020,Appleton Ave. Community Organization,Appleton Ave. Pollinator Corridor,Multiple Front Yard Gardens,18,1250.00,92,None,Private Residential Property,14 Appleton Avenue,M6E 3A4,None,Davenport,9,None,None,None,MULTIPOINT (-79.43796 43.67956)
1,2,2,2020,Centre for Immigrant and Community Services (C...,The ENRICH Pollinator Garden,Communal Garden,4,114.25,128,None,Centre for Immigrant and Community Services,2330 Midland Avenue,M1S 5G5,None,Scarborough-Agincourt,22,None,None,None,MULTIPOINT (-79.27601 43.77839)
2,3,3,2020,Chester School Pollinator Garden Team,Chester School Pollinator Garden,School Learning Garden,1,34.00,57,None,Chester Elementary School,115 Gowan Avenue,M4K 2E4,None,Toronto-Danforth,14,None,None,None,MULTIPOINT (-79.35171 43.6876)
3,4,4,2020,CSPC - Transfiguration,Transfiguration Pollinator Garden,School Learning Garden,1,22.00,7,None,Transfiguration of Our Lord Catholic School,55 Ludstone Drive,M9R 2J2,None,Etobicoke Centre,2,None,None,None,MULTIPOINT (-79.56287 43.69153)
4,5,5,2020,Danforth Gardens Neighbourhood Association,Danforth Gardens Neighbourhood Pollinator Corr...,Multiple Front Yard Gardens,7,28.00,120,None,Private Residential Property,64 Clapperton Avenue,M1L 4K7,None,Scarborough Southwest,20,None,None,None,MULTIPOINT (-79.27635 43.71211)


In [2]:
len(data)

149

In [3]:
years = data['YEAR_FUNDED'].unique()
print(years)

[2020 2021 2022 2023]


In [6]:
print("Columns with null values:\n")
print(data.isnull().sum()[data.isnull().sum() > 0])

# If no null values are found
if data.isnull().sum().sum() == 0:
    print("\nNo null values found in the dataset.")

Columns with null values:

Series([], dtype: int64)

No null values found in the dataset.


In [9]:
data['WARD_NAME'].value_counts()

WARD_NAME
Parkdale-High Park          16
Toronto-Danforth            13
Toronto Centre              11
Davenport                   10
Beaches-East York           10
University-Rosedale          9
Scarborough Southwest        9
Scarborough-Guildwood        7
Etobicoke-Lakeshore          6
Toronto-St. Paul's           6
Humber River-Black Creek     6
Don Valley East              5
Don Valley West              5
Scarborough-Agincourt        5
York South-Weston            4
Spadina-Fort York            4
Scarborough-Rouge Park       4
Etobicoke Centre             4
York Centre                  3
Eglinton-Lawrence            3
Etobicoke North              3
Etobicoke-North              2
Willowdale                   2
Scarborough Centre           1
Don Valley North             1
Name: count, dtype: int64